### Import necessary packages

In [ ]:
%load_ext autoreload
%autoreload 2
import pyreadstat
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
import scipy
import pandas as pd
import seaborn as sns
from scipy.stats import norm, bernoulli
import matplotlib.pyplot as plt
import os, sys
import matplotlib.patheffects as pe
from utils import make_width_coverage_plot, make_budget_plot
import warnings; warnings.simplefilter('ignore')

### Import Pew ATP Wave 79 dataset

The dataset is available at: https://www.pewresearch.org/science/dataset/american-trends-panel-wave-79/

In [ ]:
data, meta = pyreadstat.read_sav("pew/ATPW79.sav")

### Problem setup

Specify estimand of interest (average approval of Biden's messaging or average approval of Trump's messaging) and compute ground-truth value of the estimand. Split data into labeled and unlabeled subsets. Specify range of budgets in fractional form $n_b/n$.

In [ ]:
question = "ELECTBIDENMSSG_W79" # can choose ELECTBIDENMSSG_W79 or ELECTTRUMPMSSG_W79 
idx_keep = np.where(data[question] != 99)[0]
Y_all = data[question].to_numpy()[idx_keep] < 2.5
X_all = data[['F_PARTYSUM_FINAL', 'COVIDFOL_W79','COVIDTHREAT_a_W79','COVIDTHREAT_b_W79','COVIDTHREAT_c_W79', 'COVIDTHREAT_d_W79','COVIDMASK1_W79', 'COVID_SCI6E_W79', 'F_EDUCCAT', 'F_AGECAT']].to_numpy()[idx_keep]
theta_true = np.mean(Y_all)
X_train, X, y_train, Y = train_test_split(X_all, Y_all*1, test_size=0.5)

In [ ]:
budgets = np.linspace(0.005, 0.2, 20)

### Train XGBoost model

Train XGBoost model on labeled data and compute model uncertainty for unlabeled instances.

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
tree = xgb.train({'eta': 0.001, 'max_depth': 5, 'objective': 'reg:logistic'}, dtrain, 3000)
Yhat = tree.predict(xgb.DMatrix(X))
uncertainty = np.minimum(Yhat, 1-Yhat)

### Tune mixing parameter $\tau$

The sampling rule is $\pi^{(\tau)}(x) = (1-\tau) \cdot \hat \eta \cdot u(x) + \tau \cdot \pi^{\mathrm{unif}}(x)$. The following code tunes $\tau$ based on training data, finding one value of $\tau$ per budget size.

In [ ]:
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=0.2)
taus = np.linspace(0.01, 0.99, 100) # range of taus that we're searching over

dtrain = xgb.DMatrix(X_train1, label=y_train1)
tree_tuning = xgb.train({'eta': 0.001, 'max_depth': 5, 'objective': 'reg:logistic'}, dtrain, 3000)
Yhat_train = tree_tuning.predict(xgb.DMatrix(X_train2))
uncertainty_train = np.minimum(Yhat_train, 1-Yhat_train)

tau_opts = np.zeros(len(budgets)) # finds one tau per budget

for i in range(len(budgets)):
    budget = budgets[i]
    eta = budget / np.mean(uncertainty_train)
    min_var = np.inf
    for tau in taus:
        probs = np.clip((1-tau)*eta*uncertainty_train + budget*tau,0,1)
        var_tau = np.mean((y_train2 - Yhat_train)**2/probs)
        if var_tau < min_var:
            tau_opts[i] = tau
            min_var = var_tau

### Main experiment

Forms dataframe ```df``` with experiment results. The columns in the dataframe are:

- ```lb``` - interval lower bound

- ```ub``` - interval upper bound

- ```interval width``` - equal to ```ub``` - ```lb```

- ```coverage``` - 0/1 indicator of whether or not interval covered target

- ```estimator``` - one of ```classical```, ```uniform```, or ```active```

- ```$n_b$``` - budget size 

In [ ]:
# parameters of experiment
n = len(Y)
num_trials = 1000
alpha = 0.1

In [ ]:
results = []
columns = ["lb", "ub", "interval width", "coverage", "estimator","$n_b$"]
temp_df = pd.DataFrame(np.zeros((3,len(columns))), columns=columns)

for j in range(len(budgets)):
    budget = budgets[j]
    tau = tau_opts[j]
    eta = budget / np.mean(uncertainty)
    probs = np.clip((1-tau)*eta*uncertainty + tau*budget, 0.0, 1.0)

    for i in range(num_trials):
        xi = bernoulli.rvs(probs)
        pointest_active = np.mean(Yhat + (Y - Yhat)*xi/probs)
        pointest_active_std = np.std(Yhat + (Y - Yhat)*xi/probs)/np.sqrt(n)
        width_active = norm.ppf(1-alpha/2)*pointest_active_std 
        coverage_active = (theta_true >= pointest_active - width_active)*(theta_true <= pointest_active + width_active)   
        temp_df.loc[0] = pointest_active - width_active, pointest_active + width_active, 2*width_active, coverage_active, "active", int(budget*n)

        xi_unif = bernoulli.rvs([budget]*n)
        pointest_unif = np.mean(Yhat + (Y - Yhat)*xi_unif/budget)
        pointest_unif_std = np.std(Yhat + (Y - Yhat)*xi_unif/budget)/np.sqrt(n)
        width_unif = norm.ppf(1-alpha/2)*pointest_unif_std
        coverage_unif = (theta_true >= pointest_unif - width_unif)*(theta_true <= pointest_unif + width_unif)
        temp_df.loc[1] = pointest_unif - width_unif, pointest_unif + width_unif, 2*width_unif, coverage_unif, "uniform", int(budget*n)

        pointest_class = np.mean(Y*xi_unif/budget)
        pointest_class_std = np.std(Y*xi_unif/budget)/np.sqrt(n)
        width_class = norm.ppf(1-alpha/2)*pointest_class_std
        coverage_class = (theta_true >= pointest_class - width_class)*(theta_true <= pointest_class + width_class)
        temp_df.loc[2] = pointest_class - width_class, pointest_class + width_class, 2*width_class, coverage_class, "classical", int(budget*n)        
        
        results += [temp_df.copy()]
df = pd.concat(results,ignore_index=True)

### Plot coverage and interval width

In [ ]:
make_width_coverage_plot(df, "approval rate", "widths_and_coverage_pew79_biden_batch.pdf", theta_true, n_l=200, n_u=800, n_example_ind=3)

### Plot budget saved

In [ ]:
make_budget_plot(df, "Post-election survey research (Biden)", "budget_pew79_biden_batch.pdf")